In [13]:
from operator import itemgetter
from pathlib import Path
import vectorbt as vbt
from hydra.PriceLoader import load_prices

def load_portfolio(name):
    output_dir = Path.cwd().joinpath('..', 'output')
    filepath = output_dir.joinpath(name)
    return vbt.Portfolio.load(filepath)

def parsedatetime(dt):
    return pd.to_datetime(dt)
    # return datetime.strptime(dt, '%Y-%m-%d %H:%M')

def data_extrapolator(dataset, figure, trace, fields):
    def slicer(start, end):
        start_date = parsedatetime(start)
        end_date = parsedatetime(end)
        delta = end_date - start_date
        delta_m = delta.total_seconds() / 60
        if delta_m < 500:
            interval = 1
        elif delta_m < 500 * 5:
            interval = 5
        elif delta_m < 500 * 15:
            interval = 15
        elif delta_m < 500 * 60:
            interval = 60
        elif delta_m < 500 * 720:
            interval = 720
        else:
            interval = 1440

        fakestart = start_date - timedelta(minutes=interval * 250)
        fakeend = end_date + timedelta(minutes=interval * 250)
        precise_date = dataset[interval].loc[fakestart:fakeend]
        orig_date = dataset[1440]
        the_date = pd.concat([orig_date.query('index < @fakestart or index > @fakeend'), precise_date]).sort_index()
        # f = open('log.txt', "a")
        # f.write(f"[{utils.now()}] {start} {end} {delta} {delta_m} {interval} \n{precise_price} \n {fakestart} {fakeend} \n{the_price.loc[fakestart:fakeend]}\n")
        # f.close()
        with figure.batch_update():
            for key, val in fields.items():
                try:
                    value = getattr(the_date, val)
                except:
                    value = the_date[val]

                setattr(trace, key, value)


    return slicer




In [26]:
pair='XBTUSD'
start_date = "2019-05-20 00:00"
end_date = "2019-05-30 00:00"
interval=1


prices = {}
prices[1] = load_prices(pair, '../data/kraken', start_date, end_date, 1)
prices[5] = load_prices(pair, '../data/kraken', start_date, end_date, 5)
prices[15] = load_prices(pair, '../data/kraken', start_date, end_date, 15)
prices[60] = load_prices(pair, '../data/kraken', start_date, end_date, 60)
prices[720] = load_prices(pair, '../data/kraken', start_date, end_date,720)
prices[1440] = load_prices(pair, '../data/kraken', start_date, end_date, 1440)

In [27]:
from datetime import datetime

def str_to_datetime(str) -> datetime:
    return (
        datetime.strptime(str, "%Y-%m-%d %H:%M:%S")
        if not isinstance(str, datetime)
        else str
    )
orders = [["2019-05-20 05:05:00", "Direction.BUY"], ["2019-05-20 05:42:00", "Direction.SELL"], ["2019-05-20 05:47:00", "Direction.BUY"], ["2019-05-20 06:19:00", "Direction.SELL"], ["2019-05-20 06:24:00", "Direction.BUY"], ["2019-05-20 07:44:00", "Direction.SELL"], ["2019-05-20 16:50:00", "Direction.BUY"], ["2019-05-20 16:58:00", "Direction.SELL"], ["2019-05-20 17:03:00", "Direction.BUY"], ["2019-05-20 17:19:00", "Direction.SELL"], ["2019-05-20 17:24:00", "Direction.BUY"], ["2019-05-20 17:57:00", "Direction.SELL"], ["2019-05-20 18:05:00", "Direction.BUY"], ["2019-05-20 18:35:00", "Direction.SELL"], ["2019-05-20 18:43:00", "Direction.BUY"], ["2019-05-20 19:16:00", "Direction.SELL"], ["2019-05-20 19:18:00", "Direction.BUY"], ["2019-05-20 20:19:00", "Direction.SELL"], ["2019-05-20 20:25:00", "Direction.BUY"], ["2019-05-20 21:15:00", "Direction.SELL"], ["2019-05-20 21:18:00", "Direction.BUY"], ["2019-05-20 21:38:00", "Direction.SELL"], ["2019-05-20 21:41:00", "Direction.BUY"]]

orders = [
    {
      "timestamp":str_to_datetime(order[0]), 
      "order": True if order[1] == "Direction.BUY" else False,
      "price": prices[1].loc[str_to_datetime(order[0])]['open']
    } for order in orders
]


[{'timestamp': datetime.datetime(2019, 5, 20, 5, 5), 'order': True, 'price': 8015.0}, {'timestamp': datetime.datetime(2019, 5, 20, 5, 42), 'order': False, 'price': 7982.3}, {'timestamp': datetime.datetime(2019, 5, 20, 5, 47), 'order': True, 'price': 7960.3}, {'timestamp': datetime.datetime(2019, 5, 20, 6, 19), 'order': False, 'price': 8004.0}, {'timestamp': datetime.datetime(2019, 5, 20, 6, 24), 'order': True, 'price': 7995.1}, {'timestamp': datetime.datetime(2019, 5, 20, 7, 44), 'order': False, 'price': 7949.2}, {'timestamp': datetime.datetime(2019, 5, 20, 16, 50), 'order': True, 'price': 7858.4}, {'timestamp': datetime.datetime(2019, 5, 20, 16, 58), 'order': False, 'price': 7860.1}, {'timestamp': datetime.datetime(2019, 5, 20, 17, 3), 'order': True, 'price': 7846.1}, {'timestamp': datetime.datetime(2019, 5, 20, 17, 19), 'order': False, 'price': 7850.1}, {'timestamp': datetime.datetime(2019, 5, 20, 17, 24), 'order': True, 'price': 7845.3}, {'timestamp': datetime.datetime(2019, 5, 20, 

In [28]:
import plotly.graph_objects as go
import pandas as pd
import hydra.utils as utils
from datetime import datetime, timedelta


candlestick = go.Candlestick(
        x=prices[1440].index,
        open=prices[1440]["open"],
        high=prices[1440]["high"],
        low=prices[1440]["low"],
        close=prices[1440]["close"],
        hoverinfo="all",
    )

buy_date = [order['timestamp'] for order in orders if order['order']]
sell_date = [order['timestamp'] for order in orders if not order['order']]
buy_price = [order['price'] for order in orders if order['order']]
sell_price = [order['price'] for order in orders if not order['order']]

# print(portfolio.total_profit())
# print(portfolio.orders.buy.records)
# buy_idx = portfolio.orders.buy.records['idx']
# buy_date = prices[1].index[buy_idx]
# buy_price = portfolio.orders.buy.records['price']
# buy_size = portfolio.orders.buy.records['size']
# buy_text = [f"{size} | {price}" for size, price in zip(buy_size, buy_price)]

# sell_idx = portfolio.orders.sell.records['idx']
# sell_date = prices[1].index[sell_idx]
# sell_price = portfolio.orders.sell.records['price']
# sell_size = portfolio.orders.sell.records['size']
# sell_text = [f"{size} | {price}" for size, price in zip(sell_size, sell_price)]


buys_trace = go.Scattergl(
    name="Buys",
    marker_color="green",
    x=buy_date,
    y=buy_price,
    mode="markers",
    marker_line_width=2,
    marker_size=10,
)

sells_trace = go.Scattergl(
    name="Sells",
    marker_color="red",
    x=sell_date,
    y=sell_price,
    mode="markers",
    marker_line_width=2,
    marker_size=10,
)


figure = go.FigureWidget(
    data=[candlestick, buys_trace, sells_trace],
    # data=[candlestick],
    layout=go.Layout(
        title=dict(
            text='FLYBABYFLY'
        ),
        barmode='overlay',
        yaxis={'fixedrange': False},
        yaxis2=dict(
            fixedrange=False,
            domain=[0,1],
        )
    )
)
cs = figure.data[0]



update_candlestick = data_extrapolator(prices, figure, cs, {
    "x": "index",
    "open": "open",
    "high": "high",
    "low": "low",
    "close": "close"
})

update_candlestick(start_date,end_date)
def handler(obj, xrange):
    [start, end] = xrange.range
    update_candlestick(start, end)
figure.layout.on_change(handler , 'xaxis')
figure




TypeError: unhashable type: 'list'